<a href="https://www.kaggle.com/code/rishitratan5/age-prediction-of-chest-x-ray-images-using-resnet?scriptVersionId=161500904" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Age Prediction of Patients by Chest X-rays

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms import ToTensor, Compose, Resize, Grayscale, Normalize, Lambda
import os
import torchvision.transforms as T
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from torch import optim
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device('cuda')

In [3]:
# Define the training image directory and annotations csv
train_img_dir = '/kaggle/input/minijsrtageprediction/XPAge01_RGB/XP/JPGs/'
train_annotations_file = '/kaggle/input/minijsrtageprediction/XPAge01_RGB/XP/trainingdata.csv'

# Define the testing image directory and annotations csv
test_img_dir = '/kaggle/input/minijsrtageprediction/XPAge01_RGB/XP/JPGs/'
test_annotations_file = '/kaggle/input/minijsrtageprediction/XPAge01_RGB/XP/testdata.csv'

# Define the output classes here
# num_classes = 100

In [4]:
df = pd.read_csv(test_annotations_file)
df['age'].max()

89

In [5]:
# Define a custom class for dataset 
class CustomDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.img_labels.iloc[idx, 0]
        image = read_image(img_path)
        image = T.ToPILImage() (image)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
# Definining the resnet18 model
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
class Resnet18Classifier(nn.Module):
    def __init__(self):
        super(Resnet18Classifier, self).__init__()
        self.resnet18 = resnet18(pretrained=True).eval()
        num_ftrs = self.resnet18.fc.out_features
        self.fc = nn.Linear(num_ftrs, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.fc(self.resnet18(x))

In [7]:
# Defining the transform for loading the data
transform = Compose([
    ToTensor(),
    Lambda(lambda x: x.repeat(3,1,1)),
    Resize(256)
])

# Defining the training dataset and data loader
train_dataset = CustomDataset(
    annotations_file=train_annotations_file,
    img_dir=train_img_dir,
    transform=transform,
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Defining the testing dataset and data loader
test_dataset = CustomDataset(
    annotations_file=test_annotations_file,
    img_dir=test_img_dir,
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [8]:
# Initializing the resnet18 classifier with number of output models
resnet_18_classifier = Resnet18Classifier().to(device)

# Defining the loss_function and the optimizer
loss_func = nn.L1Loss()
optimizer = optim.AdamW(resnet_18_classifier.parameters(), lr = 0.01)
resnet_18_classifier

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 48.8MB/s]


Resnet18Classifier(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [9]:
# Training the model
num_epochs = 100
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_18_classifier(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training finished")

Epoch [1/100], Loss: 79.6564
Epoch [2/100], Loss: 33.7119
Epoch [3/100], Loss: 28.8661
Epoch [4/100], Loss: 13.6859
Epoch [5/100], Loss: 28.4021
Epoch [6/100], Loss: 17.0776
Epoch [7/100], Loss: 58.8713
Epoch [8/100], Loss: 119.8967
Epoch [9/100], Loss: 12.5335
Epoch [10/100], Loss: 19.4335
Epoch [11/100], Loss: 12.8801
Epoch [12/100], Loss: 13.0153
Epoch [13/100], Loss: 13.0299
Epoch [14/100], Loss: 14.2978
Epoch [15/100], Loss: 11.6814
Epoch [16/100], Loss: 16.8158
Epoch [17/100], Loss: 12.4375
Epoch [18/100], Loss: 16.5389
Epoch [19/100], Loss: 17.8750
Epoch [20/100], Loss: 12.3015
Epoch [21/100], Loss: 18.4701
Epoch [22/100], Loss: 13.4871
Epoch [23/100], Loss: 13.0680
Epoch [24/100], Loss: 13.0657
Epoch [25/100], Loss: 13.4167
Epoch [26/100], Loss: 13.1368
Epoch [27/100], Loss: 15.6793
Epoch [28/100], Loss: 10.1878
Epoch [29/100], Loss: 10.4531
Epoch [30/100], Loss: 8.1250
Epoch [31/100], Loss: 11.8750
Epoch [32/100], Loss: 12.4375
Epoch [33/100], Loss: 12.2409
Epoch [34/100], Los

In [10]:
# Testing the model
test_loss = 0.0
with torch.no_grad():
    resnet_18_classifier.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet_18_classifier(images)
        loss = loss_func(outputs, labels)
        test_loss += loss.item()
        

accuracy = test_loss / len(test_loader)
print(f"Test Accuracy for age: {accuracy:.2f}")

Test Accuracy for age: 11.86
